In [ ]:
from gw_grb_correlation.Fermi.download_data_functions import download_file


In [ ]:
# List of detectors you want
detectors = ['n1', 'n2', 'n5']

# Base URL parts
url_tte = "https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/2017/bn170817529/current/"
filename_tte = "glg_tte_{}_bn170817529_v00.fit"

# Loop to download each file
for det in detectors:
    url = url_tte + filename_tte.format(det)
    filename = filename_tte.format(det)
    download_file(url, filename)

url_bcat = "https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/2017/bn170817529/current/glg_bcat_all_bn170817529_v01.fit"
filename_bcat = "glg_bcat_bn170817529_v00.fit"
download_file(url_bcat, filename_bcat)

In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
def extract_tte_data(tte_file):
    
    with fits.open(tte_file) as hdul:
        df = pd.DataFrame(columns=['TIME'])
        df['TIME'] = hdul['EVENTS'].data['TIME']
        return df

def extract_count_rate(df, bins=256):
    
    # Create time bins
    time = df['TIME']
    bin_edges = np.linspace(time.min(), time.max(), bins)
    bin_size = bin_edges[1] - bin_edges[0]
    digitized = np.digitize(time, bin_edges)
    
    # Calculate count rate in each bin
    count_rate = [np.sum(digitized == i) / bin_size for i in range(1, len(bin_edges))]

    return bin_edges[1:], count_rate

def extract_bcat_data(tte_file):
    
    with fits.open(tte_file) as hdul:
        header = hdul['PRIMARY'].header
        return [header['TSTART'], header['TSTOP']]

# Extracting TTE data
tte_data = extract_tte_data(filename_tte.format('n2'))
time, count_rate = extract_count_rate(tte_data)

# Extracting BCAT data
bcat_data = extract_bcat_data(filename_bcat)
print(f"start time: {bcat_data[0]}")
print(f"stop time: {bcat_data[1]}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming time and count_rate are already defined
# Calculate the baseline of the count rate
Baseline = np.average(count_rate)

# Plot the count rate
plt.plot(time, count_rate, label='Count Rate', color='blue')

# Plot the baseline as a horizontal line
plt.axhline(y=Baseline, color='red', linestyle='--', label='Baseline')

# Plot vertical lines for tstart and tstop
tstart, tstop = bcat_data
plt.axvline(x=tstart, color='green', linestyle='--', label='TSTART')
plt.axvline(x=tstop, color='orange', linestyle='--', label='TSTOP')

# Add labels and title
plt.xlabel('Time (s)')
plt.ylabel('Count Rate (counts/s)')
plt.title('Count Rate vs Time for TTE Data')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Ensure inputs are NumPy arrays
time = np.asarray(time)
count_rate = np.asarray(count_rate)

# Mask for the time window
mask = (time >= tstart) & (time <= tstop)
# Time and counts in that window
time_window = time[mask]
counts_window = count_rate[mask]

# Subtract the baseline from the counts
excess_counts = counts_window - Baseline

# Optional: Set any negative values to 0 (if you only want excess above baseline)
excess_counts = np.clip(excess_counts, a_min=0, a_max=None)

# Estimate total excess counts using the trapezoidal rule
# This integrates the rate over time to get counts
total_excess_counts = np.trapz(excess_counts, time_window)

print(f"Total excess counts above baseline in [{tstart}, {tstop}] = {total_excess_counts:.2f}")


In [ ]:
import re
def calculate_excess_counts(tstart, tstop, filename, bins =256):
    """
    Calculate the excess counts in the time window [tstart, tstop] above the baseline.
    """

    # Extract TTE data
    with fits.open(filename) as hdul:
        tte_data = pd.DataFrame(columns=['TIME'])
        tte_data['TIME'] = hdul['EVENTS'].data['TIME']

        # Extract ID from the filename
        id = re.search(r'bn\d{9}', hdul[0].header['FILENAME'])
        id = id.group(0) if id else ""
        
        # Extract detector name from the filename
        detector = re.search(r"glg_tte_(\w+)_bn", hdul[0].header['FILENAME']).group(1)
    
    # Create time bins
    time_bin = tte_data['TIME']
    bin_edges = np.linspace(time_bin.min(), time_bin.max(), bins)
    bin_size = bin_edges[1] - bin_edges[0]
    digitized = np.digitize(time_bin, bin_edges)
    time = bin_edges[1:]
    
    # Calculate count rate in each bin
    count_rate = [np.sum(digitized == i) / bin_size for i in range(1, len(bin_edges))]

    # Ensure inputs are NumPy arrays
    time = np.asarray(time)
    count_rate = np.asarray(count_rate)

    # Calculate the baseline of the count rate
    Baseline = np.average(count_rate)

    # Mask for the time window
    mask = (time >= tstart) & (time <= tstop)

    # Time and counts in that window
    time_window = time[mask]
    counts_window = count_rate[mask]

    # Subtract the baseline from the counts
    excess_counts = counts_window - Baseline

    # Estimate total excess counts using the trapezoidal rule
    # This integrates the rate over time to get counts
    total_excess_counts = np.trapz(excess_counts, time_window)

    return (id, detector, total_excess_counts)

In [ ]:
# Example usage
tstart, tstop = bcat_data   # Use the tstart and tstop from the BCAT data
# Calculate excess counts in the specified time window for each detector
detectors = ['n1', 'n2', 'n5']
for detector in detectors:
    filename = filename_tte.format(detector)
    excess_counts = calculate_excess_counts(tstart, tstop, filename)
    print(excess_counts[0])  # ID
    print(excess_counts[1])  # Detector
    print(f"Total excess counts for {detector} in [{tstart}, {tstop}] = {excess_counts[2]:.2f}")


In [ ]:
import numpy as np
import pandas as pd
# Load bcat_data from the specified file (time_data.npy)
file_path = f'./fermi_data/time/time_data.npy'
bcat_data = pd.DataFrame(np.load(file_path, allow_pickle=True))
bcat_data.columns = ['ID', 'TSTART', 'TSTOP', 'T90', 'DATE']
print(bcat_data.head())

In [ ]:
from astropy.io import fits
import re
def extract_fits_excess_photon_data(bcat_data, filename, bins =256):
    """
    Calculate the excess counts in the time window [tstart, tstop] above the baseline.
    """
    
    # Extract TTE data
    with fits.open(filename) as hdul:
        tte_data = pd.DataFrame(columns=['TIME'])
        tte_data['TIME'] = hdul['EVENTS'].data['TIME']

        # Extract ID from the filename
        id = re.search(r'bn\d{9}', hdul[0].header['FILENAME'])
        id = id.group(0) if id else ""
        
        # Extract tstart and tstop from bcat_data
        target = bcat_data[bcat_data['ID'] == id]
        if target.empty:
            return (id, detector, 0)
        tstart = target['TSTART'].iloc[0]  # Start time of the burst   
        tstop = target['TSTOP'].iloc[0]    # Stop time of the burst

        # Extract detector name from the filename
        detector = re.search(r"glg_tte_(\w+)_bn", hdul[0].header['FILENAME']).group(1)
    
    # Create time bins
    time_bin = tte_data['TIME']
    bin_edges = np.linspace(time_bin.min(), time_bin.max(), bins)
    bin_size = bin_edges[1] - bin_edges[0]
    digitized = np.digitize(time_bin, bin_edges)
    time = bin_edges[1:]
    
    # Calculate count rate in each bin
    count_rate = [np.sum(digitized == i) / bin_size for i in range(1, len(bin_edges))]

    # Ensure inputs are NumPy arrays
    time = np.asarray(time)
    count_rate = np.asarray(count_rate)

    # Calculate the baseline of the count rate
    Baseline = np.average(count_rate)

    # Mask for the time window
    print(f"tstart: {tstart}, tstop: {tstop}")
    mask = (time >= tstart) & (time <= tstop)

    # Time and counts in that window
    time_window = time[mask]
    counts_window = count_rate[mask]

    # Subtract the baseline from the counts
    excess_counts = counts_window - Baseline

    # Estimate total excess counts using the trapezoidal rule
    # This integrates the rate over time to get counts
    total_excess_counts = np.trapz(excess_counts, time_window)

    return (id, detector, total_excess_counts)

In [1]:
from gw_grb_correlation.Fermi.tte_data import preprocess_tte_data
preprocess_tte_data(2025, 2026)

Total files downloaded: 77


C:\Users\colon\OneDrive\桌面\TheoryWork\group_project_2025\gw_grb_correlation\Fermi\tte_data.py:176: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(new_data, columns=columns)], ignore_index=True)


Processed and saved data for 2025, detector n0
(77, 3)


Total files downloaded: 77
Processed and saved data for 2025, detector n1
(154, 3)


Total files downloaded: 77
Processed and saved data for 2025, detector n2
(231, 3)


...

Total files downloaded: 77
Processed and saved data for 2025, detector na
(847, 3)


Total files downloaded: 77
Processed and saved data for 2025, detector nb
(924, 3)


Total files downloaded: 77
Processed and saved data for 2025, detector b0
(1001, 3)


Total files downloaded: 77
Processed and saved data for 2025, detector b1
(1078, 3)


,ID,n0_PH_CNT,n1_PH_CNT,n2_PH_CNT,n3_PH_CNT,n4_PH_CNT,n5_PH_CNT,n6_PH_CNT,n7_PH_CNT,n8_PH_CNT,n9_PH_CNT,na_PH_CNT,nb_PH_CNT,b0_PH_CNT,b1_PH_CNT
0,bn250101968,-16.682350,-385.652937,-770.835314,1787.982316,2294.523560,1568.870623,1706.711801,2089.547057,1076.894118,753.876476,-1167.317648,1765.311761,322.729422,1295.058802
1,bn250102943,4819.390236,6008.113770,13172.909520,11391.499808,14196.042971,31313.798313,6726.129376,3482.615649,-7753.037359,6799.433469,12661.125313,732.701971,-2922.333305,1136.809824
2,bn250103415,3016.366606,-2139.866667,-3909.825410,10002.156986,10654.652754,3692.582339,6139.182432,5695.274464,3088.343142,549.341192,-4091.427381,1985.525542,1915.370624,5192.927328
3,bn250107210,5356.252922,4892.641276,2538.994096,7373.652985,3968.888176,858.488254,-3767.041176,-4764.494002,-3361.988301,-5204.882296,-6611.082378,-8769.570402,1635.682370,-685.770587
4,bn250107482,54.525490,148.882356,523.205869,236.898039,48.088235,572.149020,225.845098,333.829421,-157.074510,159.441179,330.225490,62.264706,87.715686,553.627451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,bn250331949,5054.117733,4578.539197,3238.686253,3668.931285,1058.931363,2472.009747,7362.509892,6967.264586,6436.519589,9779.039026,6805.137138,9611.088080,12105.039277,11708.892035
69,bn250402964,-3608.300036,-7132.000000,-9553.400036,2791.900062,10363.500032,-2390.099941,2661.099976,2422.000045,4716.099911,4572.300032,5450.900127,4622.400087,-2917.199980,621.499996
70,bn250403636,-227.400003,2072.823499,6317.047191,-5609.958787,-10703.394315,-2745.629345,-4627.476424,-10812.229484,-9095.176471,5525.135353,11307.017560,895.211746,2056.058858,959.741172
71,bn250404597,33961.215110,40458.881684,23461.745059,6931.353033,4430.862839,9343.852734,5163.637177,-5116.754883,-1102.254865,31798.244454,15366.608198,4093.823572,1906.745109,3227.725530
